# ESP32 Mesh Network and Communication Plan
> Notes on mesh communication patterns, pub/sub architecture, and wireless protocols using ESP32.

- keywords: ['esp32', 'mesh', 'bluetooth', 'mqtt', 'esp-now', 'access-point', 'wifi', 'iot', 'microcontroller']

## Overview

A single ESP32 connects via Bluetooth to your phone, and the nodes, possibly multiple, communicate via a mesh network. They message each other using a protocol called ESP-NOW. To establish ESP-NOW links, the nodes broadcast their presence using their soft-AP functionality, allowing them to exchange MAC addresses.


### Mesh Behavior

- A node first scans for an existing mesh network. If none is found, it opens itself for Bluetooth connections.
- If part of a mesh, the node creates a Soft Access Point (AP) and starts broadcasting its presence.
- The ESP32 creates an AP with both its SSID and password starting with ESP32.
- Each node maintains a routing table containing the MAC addresses of all nodes in its subnetwork (including itself), partitioned into subtables for each child node’s subnetwork.
- Any addition or removal of nodes triggers a user notification.
- Nodes can mesh across networks owned by different users and share information by scanning for other APs and reporting them to their (gateway) node.


## Communication Rules

1. Only admin may broadcast.
2. Gateway sends to children.
3. Children do not message siblings.
4. One relative per broadcast; prioritize gateway.
5. Children deliver inbound messages to gateway.
6. Gateway delivers data to admin, return to step 1.

* Broadcasts limited by hop count (`#hops`) managed by gateway.
* Gateways broadcast user commands.
* Leafs unicast out, not to each other.
* Users config determins what messages to relay from others.
* Pub/sub design allows routing and optional replication of messages.
* Peers broadcast to all peers `FF:FF:FF:FF`
* Set peer MAC address -> broadcast via shared MAC. 


### Communication Stack

* Command → STA → `WifiMacAddr()`
* Satellite → AP → `WifiSoftAPMacAddr()`
* Channels: 0–14, 2.4GHz (802.11n)
* Max paired devices: 20
* Circumvent unicasting by using common MAC addresses.

## Code Overview

### Key Variables

* `Service`, `Characteristic`
* `EspOwnerName`, `EspDeviceName`
* `Pin` (e.g., 13, 2)
* `LED` #
* `SSIDConnectorNamePrefix`

### Event Pipeline

* `MidiEvent()` → keypress DJ pad → `AudioEvent()`
* `AudioEvent()` → `Meyda()`
* `Meyda()` → `LightEvent()`
* `LightEvent()` → `audioFX(userName, [colors, duration, variation, intensity, transition])`
* Slider knobs adjust: Volume, High, Mid, Low
* Start/Stop functionality with display

## Initialization & Loop

```python
# Pseudocode
on_start:
  load settings from storage
  try connect to preferred network
  except: scan_or_establish_default_network()

default_network_loop():
  if disconnected: scan_or_establish_default_network()
```

## Setup Steps

1. `ConfigDeviceAP(mac=ssid, pass=1234)` — prints confirmation.
2. `RegisterCallbacks(msg_sent, msg_received)` — handles writing to pins.
3. `InitESPNow(on_msg_sent, on_msg_received)`:

   * `strip.begin()`
   * `strip.show()`
   * BLEESP Setup:

     * Create Service and Characteristic.
     * Register Callbacks.
     * Set Value.
     * Start BLE.

### Loop Logic

1. **Scan for leaf nodes**: `WiFi.scanNetwork()` and for each AP named `espname`, add to list.
2. **Manage leaf nodes**: Check pairing with nodes named `espname`.
3. **Send Data**
4. **Command Processing**

### BLE and ESP Communication

* `OnBLEReceived`: If `leafnodes`, broadcast via ESPNow and execute command.
* `OnESPReceived`: If BLE not connected → do nothing. Otherwise, forward to BLE if valid.

### Config Device AP

* Scan for leaf nodes.

  * If node with `esp.bt` exists and not connected to user → gateway exists.
* Manage leaf nodes:

  * Try to connect via Bluetooth.
  * On success, modify Wi-Fi name to include `bt` (indicating gateway).
* `espnow_senddata(command)`:

  * If not leader → don’t send.
  * Leader-only data reading.
  * Non-leaders may send, but leaders are authoritative.

### Partition Table

- Tracks values from all different users.

```text
Group: Updated: Value
LeafID: Updated: Value
```

### Message Routing Logic and protocol

If root:  
Handle btmessage → relay outbound.  
For publicmsg:  
Run command and broadcast.  
Update timestamp.  
If command and timestamp are duplicate → ignore.  
If not root:  
Ignore btmessage.  
For publicmsg:  
Update partition table with groupname:command:timestamp.  
Ignore if not in group.  
All nodes:    
If rootnodessid does not contain espname → ignore.  